In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow==2.4.0
!pip install keras==2.4.0
!pip install kymatio
!pip install tqdm
!pip install iterative-stratification
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==2.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/sim

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import pickle
import sys
sys.path.append("drive/MyDrive/Scattering_Novo/src")
sys.path.append("drive/MyDrive/DeSpaWN-main")
# Import Data
import scipy.io as sio
from DataHandler import DataHandler
from ModelHandler import ModelHandler
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

J=10
Q=2

configs = {
    "N_GRIDS": 5, 
    "SIGNAL_BASE_LENGTH": 12800, 
    "N_CLASS": 26, 
    "USE_NO_LOAD": False, 
    "USE_HAND_AUGMENTATION": False,
    "MARGIN_RATIO": 0.15, 
    "DATASET_PATH": "drive/MyDrive/Scattering_Novo/dataset_original/Synthetic_Full_iHall.hdf5",
    "TRAIN_SIZE": 0.9,
    "FOLDER_PATH": "drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/", 
    "FOLDER_DATA_PATH": "drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/", 
    "FEATURES_FILE_NAME": "features.mat",
    "OUTPUT_CLASSIFICATION_MODELS_PATH": "drive/MyDrive/DeSpaWN-main/classification/without_data_augmentation/",
    "N_EPOCHS_TRAINING": 500,
    "FEAT_NORMALIZATION": False,
    "TRAINING_FLAG": 1,
    "PERCENTUAL": 1.0,
    "INITIAL_EPOCH": 0,
    "TOTAL_MAX_EPOCHS": 5000,
    "SNRdb": None 
}

def freeze(model, task_name='classification'):
    for layer in model.layers:
        if task_name in layer.name:
            layer.trainable = True
        else:
            layer.trainable = False

    for layer in model.layers:
        print(layer.name, layer.trainable)

    return model

def calculating_class_weights(y_true):
    '''
        Source: https://stackoverflow.com/questions/48485870/multi-label-classification-with-class-weights-in-keras
    '''
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight(class_weight='balanced', classes=[0.,1.], y=y_true[:, i])
    return weights


def reduce_dataset(X_all,ydet_all,ytype_all,yclass_all,percentual):
    import numpy as np
    max_index = int(percentual*X_all.shape[0])
    np.random.seed(100)
    index = np.random.randint(max_index,size=(max_index-1))
    X_all = X_all[index]
    ydet_all = ydet_all[index]
    ytype_all = ytype_all[index]
    yclass_all = yclass_all[index]

    return X_all,ydet_all,ytype_all,yclass_all

ngrids = configs["N_GRIDS"]
signalBaseLength = configs["SIGNAL_BASE_LENGTH"]
trainSize = configs["TRAIN_SIZE"]
folderDataPath = configs["FOLDER_DATA_PATH"]
 

folderPath = 'drive/MyDrive/Scattering_Novo/results_second_order/' + 'Hybrid2_P' + str(int(configs['PERCENTUAL']*100)) + '_J' + str(J) + '_Q' + str(Q) + '/'
#folderPath = configs["TESTS_FOLDER"] + 'Hybrid_P' + str(int(configs["PERCENTUAL"]*100)) + '_J' + str(J) + '_Q' + str(Q) + '/'
folderDataPath = configs["FOLDER_DATA_PATH"]

trainSize = configs["TRAIN_SIZE"]

dict_data = pickle.load(open(folderDataPath + "data.p", "rb")) # Load data

modelHandler = ModelHandler(configs)

features_folder_name = configs["FOLDER_PATH"] + "Training_Features" 



In [ ]:
training_flag=configs["TRAINING_FLAG"]

## Load Features

In [ ]:
import scipy.io as sio


### Merging the Training Features

In [ ]:
full_training_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_training_grids2.mat"
full_testing_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_testing_grids2.mat"

training_features_folder_name = configs["FOLDER_DATA_PATH"] + "Training_Features_grids2" 

if not os.path.exists(full_training_set_path):
    #feats= np.zeros([0,14])
    print(full_training_set_path)
    number_of_training_files= 75
    try: 
      for cont in range(number_of_training_files+1): # Number of training files
        path = "features_train_grids2" + str(cont) + ".mat"

        if os.path.isfile(os.path.join(training_features_folder_name,path)):
              csv_path = os.path.join(training_features_folder_name,path)

              print(csv_path)
              #print("Extracting from " + path)
              try:
                imported = sio.loadmat(csv_path)
                imported2 = imported['arr']

                if cont==0:
                    feats = imported2
                else:
                    feats = np.append(feats,  imported2, axis=0 )
              except:
                print("Fail merging file " + path)
        else:
              print("File is not from training dataset")
    except:
      print("Training Directory does not exist")

    sio.savemat(full_training_set_path, {'arr': feats})
    print("Combined and saved!")

    print(feats.shape)

    del feats, imported2


In [ ]:
print("Training data exists? ", os.path.exists(full_training_set_path))
print("Testing data exists? ", os.path.exists(full_testing_set_path))


Training data exists?  True
Testing data exists?  True


In [ ]:
X_train = sio.loadmat(full_training_set_path)
X_train = X_train['arr']
y_train = sio.loadmat(configs["FOLDER_DATA_PATH"] + "yclass_train_.mat")
y_train = y_train['arr']


In [ ]:
print(X_train.shape)
print(y_train.shape)

(7575, 5, 10)
(7575, 5, 26)


### Merging the testing features

In [ ]:
number_of_testing_files= 8
#features_folder_name = configs["FOLDER_DATA_PATH"] + "Testing_Features" 
testing_features_folder_name = configs["FOLDER_DATA_PATH"] + "Testing_Features_grids2" 

full_testing_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_testing_grids2.mat"

if not os.path.exists(full_testing_set_path):
    try: 
      for cont in range(number_of_testing_files+1): # Number of training files
        path = "features_test_grids2" + str(cont) + ".mat"

        if os.path.isfile(os.path.join(testing_features_folder_name,path)):
              csv_path = os.path.join(testing_features_folder_name,path)
              #print("Extracting from " + path)
              try:
                imported = sio.loadmat(csv_path)
                imported2 = imported['arr']
                #imported2 = imported2.reshape([imported2.shape[1],])

                if cont==0:
                    feats = imported2
                else:
                    feats = np.append(feats,  imported2, axis=0 )
              except:
                print("Fail merging file " + path)
        else:
              print("File is not from testing dataset")
    except:
      print("Training Directory does not exist")

    sio.savemat(configs["FOLDER_PATH"] + "combined_features_testing_grids2" + ".mat", {'arr': feats})
    print("Combined and saved!")
    del feats, imported2
else:
    print("Testing data already merged.")

Testing data already merged.


In [ ]:
os.path.exists(full_training_set_path)

True

In [ ]:
print(full_testing_set_path)

X_test  = sio.loadmat(full_testing_set_path)
X_test = X_test['arr']
y_test  = sio.loadmat(configs["FOLDER_DATA_PATH"] + "yclass_test_.mat")
y_test = y_test['arr']


drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/combined_features_testing_grids2.mat


In [ ]:
print("Shape of Classification Testing Features: ", X_test.shape)
print("Shape of Classification Testing Labels: ", y_test.shape)


Shape of Classification Testing Features:  (841, 5, 10)
Shape of Classification Testing Labels:  (841, 5, 26)


## Features Normalization

In [ ]:
if configs["FEAT_NORMALIZATION"]:
  from sklearn.preprocessing import MultiLabelBinarizer
  from sklearn.preprocessing import MaxAbsScaler 
  for grid in range(5): 
    if grid==0:
      transformer = MaxAbsScaler().fit(X_test[:,grid,:])
      x_test_norm = transformer.transform(X_test[:,grid,:])
      transformer = MaxAbsScaler().fit(X_train[:,grid,:])
      X_train_norm = transformer.transform(X_train[:,grid,:])
    else:
      transformer = MaxAbsScaler().fit(X_test[:,grid,:])
      x_test_norm = np.append(x_test_norm, transformer.transform(X_test[:,grid,:]), axis=0 )
      transformer = MaxAbsScaler().fit(X_train[:,grid,:])
      X_train_norm = np.append( X_train_norm, transformer.transform(X_train[:,grid,:]), axis=0)

  x_test_norm = x_test_norm.reshape([-1,5,10])
  X_train_norm = X_train_norm.reshape([-1,5,10])

  print(x_test_norm.shape)
  print(X_train_norm.shape)

  X_test = x_test_norm
  X_train = X_train_norm

  del x_test_norm, X_train_norm

## Training

In [ ]:
def train_all_traditional_classifiers(X_train,y_train_bin,X_test,y_test_bin,source_models_folder=configs["OUTPUT_CLASSIFICATION_MODELS_PATH"]):    
    # Classification Tests
    from sklearn.datasets import make_multilabel_classification
    from sklearn.multioutput import MultiOutputClassifier
    from sklearn.metrics import multilabel_confusion_matrix
    from sklearn.metrics import classification_report

    # Logistic Regression
    from sklearn.linear_model import LogisticRegression
    LR_cls = MultiOutputClassifier(LogisticRegression()).fit(X_train, y_train_bin)
    y_pred_LR = LR_cls.predict(X_test)
    #LR_confusion = multilabel_confusion_matrix(y_test_bin, y_pred_LR)
    #LR_report = classification_report(y_test_bin, y_pred_LR)
    #print(LR_report)

    # Decision Tree
    from sklearn.tree import DecisionTreeClassifier
    TREE_cls = MultiOutputClassifier(DecisionTreeClassifier()).fit(X_train, y_train_bin)
    y_pred_TREE = TREE_cls.predict(X_test)
    #TREE_confusion = multilabel_confusion_matrix(y_test_bin, y_pred_TREE)
    #TREE_report = classification_report(y_test_bin, y_pred_TREE)
    #print(TREE_report)

    # Extra Tree (Extremely randomized trees)
    from sklearn.ensemble import BaggingClassifier
    from sklearn.tree import ExtraTreeClassifier
    extra_tree = ExtraTreeClassifier(random_state=0)
    EXTRA_cls = MultiOutputClassifier(BaggingClassifier(extra_tree, random_state=0)).fit(X_train, y_train_bin)
    y_pred_EXTRA = EXTRA_cls.predict(X_test)
    #EXTRA_confusion = multilabel_confusion_matrix(y_test_bin, y_pred_EXTRA)
    #EXTRA_report = classification_report(y_test_bin, y_pred_EXTRA)
    #print(EXTRA_report)

    # Ensemble Extra Tree
    from sklearn.ensemble import ExtraTreesClassifier
    ENSTREE_cls = MultiOutputClassifier(ExtraTreesClassifier(n_estimators=400, random_state=0)).fit(X_train, y_train_bin)
    y_pred_ENSTREE = ENSTREE_cls.predict(X_test)
    #ENSTREE_confusion = multilabel_confusion_matrix(y_test_bin, y_pred_ENSTREE)
    #ENSTREE_report = classification_report(y_test_bin, y_pred_ENSTREE)
    #print(ENSTREE_report)

    # K-Neighbors
    from sklearn.neighbors import KNeighborsClassifier
    KNN_cls = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=2)).fit(X_train, y_train_bin)
    y_pred_KNN = KNN_cls.predict(X_test)
    #KNN_confusion = multilabel_confusion_matrix(y_test_bin, y_pred_KNN)
    #KNN_report = classification_report(y_test_bin, y_pred_KNN)
    #print(KNN_report)

    # MLP - Multi-Layer Perceptron
    from sklearn.neural_network import MLPClassifier
    MLP_cls = MultiOutputClassifier(MLPClassifier(random_state=1, max_iter=500)).fit(X_train, y_train_bin)
    y_pred_MLP = MLP_cls.predict(X_test)


    # Random Forest
    from sklearn.ensemble import RandomForestClassifier
    RF_cls = MultiOutputClassifier(RandomForestClassifier(max_depth=3, random_state=0)).fit(X_train, y_train_bin)
    y_pred_RF = RF_cls.predict(X_test)
    #RF_confusion = multilabel_confusion_matrix(y_test_bin, y_pred_RF)
    #RF_report = classification_report(y_test_bin, y_pred_RF)
    #print(RF_report)

    # Ridge Classifier
    from sklearn.linear_model import RidgeClassifier
    RID_cls = MultiOutputClassifier(RidgeClassifier()).fit(X_train, y_train_bin)
    y_pred_RID = RID_cls.predict(X_test)
    #RID_confusion = multilabel_confusion_matrix(y_test_bin, y_pred_RID)
    #RID_report = classification_report(y_test_bin, y_pred_RID)
    #print(RID_report)

    # Ridge Classifier with Cross-Validation
    from sklearn.linear_model import RidgeClassifierCV
    CVRID_cls = MultiOutputClassifier(RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1])).fit(X_train, y_train_bin)
    y_pred_CVRID = CVRID_cls.predict(X_test)
    #CVRID_confusion = multilabel_confusion_matrix(y_test_bin, y_pred_CVRID)
    #CVRID_report = classification_report(y_test_bin, y_pred_CVRID)
    #print(CVRID_report)

    # Predictions 

    predictions = [y_pred_LR, y_pred_TREE, y_pred_EXTRA, y_pred_ENSTREE, y_pred_KNN, y_pred_MLP, y_pred_RF, y_pred_RID, y_pred_CVRID]

    # Saving trained models
    # Execute these code lines manually

    with open(source_models_folder + 'LR_cls.pkl', 'wb') as f:
        pickle.dump(LR_cls, f)
        
    with open(source_models_folder + 'TREE_cls.pkl', 'wb') as f:
        pickle.dump(TREE_cls, f)
        
    with open(source_models_folder + 'EXTRA_cls.pkl', 'wb') as f:
        pickle.dump(EXTRA_cls, f)
        
    with open(source_models_folder + 'ENSTREE_cls.pkl', 'wb') as f:
        pickle.dump(ENSTREE_cls, f)

    with open(source_models_folder + 'KNN_cls.pkl', 'wb') as f:
        pickle.dump(KNN_cls, f)

    with open(source_models_folder + 'MLP_cls.pkl', 'wb') as f:
        pickle.dump(MLP_cls, f)

    with open(source_models_folder + 'RF_cls.pkl', 'wb') as f:
        pickle.dump(RF_cls, f)

    with open(source_models_folder + 'RID_cls.pkl', 'wb') as f:
        pickle.dump(RID_cls, f)

    with open(source_models_folder + 'CVRID_cls.pkl', 'wb') as f:
        pickle.dump(CVRID_cls, f)
    
    return predictions

In [ ]:
if training_flag==1:    
    from numpy.lib import shape_base
    for grid in range(configs["N_GRIDS"]):
      
      output_address = configs["OUTPUT_CLASSIFICATION_MODELS_PATH"] + "grid_" + str(grid+1) + "/"
      try:
        os.mkdir(output_address)
      except:
        pass    

      # Training all models for each grid
      print("Starting to train the classifiers of grid ", (grid+1) )
      predictions = train_all_traditional_classifiers(X_train=X_train[:,grid,:], y_train_bin=y_train[:,grid,:], X_test=X_test[:,grid,:], y_test_bin=y_test[:,grid,:], source_models_folder=output_address)
      print("Models of grid " + str(grid+1) + " trained sucessfully.")


      if grid==0:
        y_pred_LR = predictions[0]
        y_pred_LR = y_pred_LR.reshape([1,y_pred_LR.shape[0],y_pred_LR.shape[1]])

        y_pred_TREE = predictions[1]
        y_pred_TREE = y_pred_TREE.reshape([1,y_pred_TREE.shape[0],y_pred_TREE.shape[1]])

        y_pred_EXTRA = predictions[2]
        y_pred_EXTRA = y_pred_EXTRA.reshape([1,y_pred_EXTRA.shape[0],y_pred_EXTRA.shape[1]])

        y_pred_ENSTREE = predictions[3]
        y_pred_ENSTREE = y_pred_ENSTREE.reshape([1,y_pred_ENSTREE.shape[0],y_pred_ENSTREE.shape[1]])

        y_pred_KNN = predictions[4]
        y_pred_KNN = y_pred_KNN.reshape([1,y_pred_KNN.shape[0],y_pred_KNN.shape[1]])

        y_pred_MLP = predictions[5]
        y_pred_MLP = y_pred_MLP.reshape([1,y_pred_MLP.shape[0],y_pred_MLP.shape[1]])

        y_pred_RF = predictions[6]
        y_pred_RF = y_pred_RF.reshape([1,y_pred_RF.shape[0],y_pred_RF.shape[1]])

        y_pred_RID = predictions[7]
        y_pred_RID = y_pred_RID.reshape([1,y_pred_RID.shape[0],y_pred_RID.shape[1]])

        y_pred_CVRID = predictions[8]
        y_pred_CVRID = y_pred_CVRID.reshape([1,y_pred_CVRID.shape[0],y_pred_CVRID.shape[1]])

        shape_LR = y_pred_LR.shape
        shape_TREE = y_pred_TREE.shape
        shape_EXTRA = y_pred_EXTRA.shape
        shape_ENSTREE = y_pred_ENSTREE.shape
        shape_KNN = y_pred_KNN.shape
        shape_MLP = y_pred_MLP.shape
        shape_RF = y_pred_RF.shape
        shape_RID = y_pred_RID.shape
        shape_CVRID = y_pred_CVRID.shape

      else:


        y_pred_LR = np.append(y_pred_LR, predictions[0].reshape(shape_LR), axis=0)

        y_pred_TREE = np.append(y_pred_TREE, predictions[1].reshape(shape_TREE), axis=0)

        y_pred_EXTRA = np.append(y_pred_EXTRA, predictions[2].reshape(shape_EXTRA), axis=0)

        y_pred_ENSTREE = np.append(y_pred_ENSTREE, predictions[3].reshape(shape_ENSTREE), axis=0)

        y_pred_KNN = np.append(y_pred_KNN, predictions[4].reshape(shape_KNN), axis=0)

        y_pred_MLP = np.append(y_pred_MLP, predictions[5].reshape(shape_MLP), axis=0)

        y_pred_RF = np.append(y_pred_RF, predictions[6].reshape(shape_RF), axis=0)

        y_pred_RID = np.append(y_pred_RID, predictions[7].reshape(shape_RID), axis=0)

        y_pred_CVRID = np.append(y_pred_CVRID, predictions[8].reshape(shape_CVRID), axis=0)
      
        # Predictions shape so far = (ngrids, nsxamples, nclass)

Starting to train the classifiers of grid  1
Models of grid 1 trained sucessfully.
Starting to train the classifiers of grid  2
Models of grid 2 trained sucessfully.
Starting to train the classifiers of grid  3
Models of grid 3 trained sucessfully.
Starting to train the classifiers of grid  4
Models of grid 4 trained sucessfully.
Starting to train the classifiers of grid  5
Models of grid 5 trained sucessfully.


## Saving Predictions

In [ ]:
predictions_path = configs["OUTPUT_CLASSIFICATION_MODELS_PATH"] + "Predictions_all_Classifiers/"


if training_flag==1:    
    Predictions = [y_pred_LR, y_pred_TREE, y_pred_EXTRA, y_pred_ENSTREE, y_pred_KNN, y_pred_MLP, y_pred_RF, y_pred_RID, y_pred_CVRID]

    try:
      os.mkdir(predictions_path)
    except:
      pass

    sio.savemat(predictions_path + "Predictions_all_Classifiers" + ".mat", {'Predictions': Predictions})

In [ ]:
Predictions = sio.loadmat(predictions_path + "Predictions_all_Classifiers" + ".mat")
Predictions = Predictions["Predictions"]

## Evaluation

In [ ]:
def make_final_predictions(Y_pred,dict_data=dict_data):
    
    Y_pred = np.transpose(Y_pred, (1, 0, 2)) # Changing axis order, to (nexamples, ngrids, nclasses)
    
    final_prediction = []
    final_groundTruth = []
    for pred, yclass, ytype in zip(Y_pred, dict_data["y_test"]["classification"], dict_data["y_test"]["type"]):
        
        prediction = np.max(pred,axis=0)
        
        #pred = bestModel.predict([np.expand_dims(xi, axis=0),np.expand_dims(xi_nd, axis=0)])
        # Essa lista "pred" tem 2 elementos. O elemento na posição 0 (pred[0]) tem shape (1,5,3), que representa a detecção de tipo de evento para cada grid. Já pred[1] tem shape (1,5,26), e representa a classificação multi-label
        ##prediction = np.max(pred[1][0],axis=0)
        # pred[1][0].shape = (5,26), e prediction é a maior predição entre os grids

        # yclass tem shape (5,26)
        groundTruth = np.max(yclass,axis=0) # groundTruth has shape (26,)

        final_prediction.append(prediction)
        final_groundTruth.append(groundTruth) 

        del yclass, ytype

        y = {}
    y["true"] = final_groundTruth.copy()
    y["pred"] = final_prediction.copy()

    return y["true"], y["pred"]



### F1 Score

#### F1 Macro:
$$
\begin{gather*}
F1_{Macro} = \frac{1}{Y} \sum_{i=1}^{Y} \frac{2 \cdot tp_i}{2 \cdot tp_i + fp_i + fn_i}
\end{gather*}
$$

#### F1 Micro:
$$
\begin{gather*}
F1_{Micro} = \frac{2 \cdot \sum_{i=1}^{Y} tp_i}{\sum_{i=1}^{Y} 2 \cdot tp_i + fp_i + fn_i}
\end{gather*}
$$

- $tp_i$: True positives classifications for appliance $i$
- $fp_i$: False positives classifications for appliance $i$
- $fn_i$: False negatives classifications for appliance $i$

In [ ]:
#from sklearn.metrics import f1_score

from sklearn.metrics import f1_score, precision_score, recall_score 
for k in range(len(Predictions)): # for each classifier
#for k in range(4): # for each classifier
    if k==0:
       y = {}
       y["true"], y["pred"] = make_final_predictions(Y_pred=Predictions[k])  
       y_true = np.array(y["true"])
       y_pred = np.array(y["pred"])
       threshold = 0.5

       f1_macro = f1_score(y_true > threshold, y_pred > threshold, average='macro')
       f1_micro = f1_score(y_true > threshold, y_pred > threshold, average='micro')

    else:
       y = {}
       y["true"], y["pred"] = make_final_predictions(Y_pred=Predictions[k])  
       y_true = np.array(y["true"])
       y_pred = np.array(y["pred"])
       threshold = 0.5

       f1_macro = np.append(f1_macro, f1_score(y_true > threshold, y_pred > threshold, average='macro') )
       f1_micro = np.append(f1_micro, f1_score(y_true > threshold, y_pred > threshold, average='micro') )

print("F1 macro: ", f1_macro)
print("F1 micro: ", f1_micro)
       


F1 macro:  [0.         0.29800857 0.26512197 0.28405482 0.1914795  0.04872303
 0.         0.         0.        ]
F1 micro:  [0.         0.35613896 0.38427948 0.4069975  0.28276072 0.06544373
 0.         0.         0.        ]


### Accuracy (ACC)

$$
\begin{gather*}
ACC_i = \frac{CCE_i}{TNE_i} \\ \\
ACC = \frac{1}{Y} \sum_{i = 1}^{Y} ACC_i
\end{gather*}
$$

- $ACC_i$: Accuracy for appliance $i$
- $CCE_i$: Load connected successfully identified
- $TNE_i$: Total of connected events

In [ ]:
threshold = 0.5

correct_on = np.zeros((26,1))
total_on = np.zeros((26,1))
correct_off = np.zeros((26,1))
total_off = np.zeros((26,1))
correct_no_event = np.zeros((26,1))
total_no_event = np.zeros((26,1))

for ytype, ytrue, ypred in zip(dict_data["y_test"]["type"], y["true"], y["pred"]):
    event_type = np.min(np.argmax(ytype, axis=1))
    if event_type == 0:
        correct_on[np.bitwise_and(ytrue > threshold, ypred > threshold)] += 1
        total_on[ytrue > threshold] += 1
    elif event_type == 1:
        correct_off[np.bitwise_and(ytrue > threshold, ypred > threshold)] += 1
        total_off[ytrue > threshold] += 1
    else:
        correct_no_event[np.bitwise_and(ytrue > threshold, ypred > threshold)] += 1
        total_no_event[ytrue > threshold] += 1

acc_on = 100 * np.average(np.nan_to_num(correct_on/total_on))
acc_off = 100 * np.average(np.nan_to_num(correct_off/total_off))
acc_no_event = 100 * np.average(np.nan_to_num(correct_no_event/total_no_event))
acc_total = 100 * np.average(np.nan_to_num((correct_on + correct_off + correct_no_event)/(total_on + total_off + total_no_event)))



<ipython-input-106-eecc4f9c870a>:23: RuntimeWarning: invalid value encountered in true_divide
  acc_off = 100 * np.average(np.nan_to_num(correct_off/total_off))
<ipython-input-106-eecc4f9c870a>:24: RuntimeWarning: invalid value encountered in true_divide
  acc_no_event = 100 * np.average(np.nan_to_num(correct_no_event/total_no_event))
